In [ ]:
# standard imports
import json
from pathlib import Path
from itertools import chain

# third party imports
import dask.dataframe as dd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# constants
DATAFRAMES = Path('./dataframes')

In [ ]:
# load hdf files
dfs = {file_.stem: dd.read_hdf(str(file_), key='df-*')
        for file_ in DATAFRAMES.glob('*.hdf')}

## Page Inits

In [ ]:
# variables for easy referencing
pi = dfs['PageInits'].compute()

In [ ]:
pi = pi.sort_values(['publisherUserId', 'request.requestedAt'])

In [ ]:
pi.head()

,browserRequest.screenHeight,browserRequest.screenPixelDepth,browserRequest.screenWidth,campaignReferrer,deviceVentileGroup,pageId,pageInitializationId,productCategoryType,productPath,publisherUserId,request.requestedAt,segmentationType
8936,960,0,1280,a88b7dcd1a9e3e17770bbaa6d7515b31a2d7e85d,10,flight.home,aa6621a4-2d8b-4c4e-be91-f0dbe23eb3d0,FLIGHTS,FLIGHTS,00001d27aa482ad2b982296c016365eed6441502,2019-10-17 09:54:25.819000+00:00,LOW_CONVERTING
4946,912,24,1368,d0941e68da8f38151ff86a61fc59f7c5cf9fcaa2,2,flight.home,ec4281fa-dd7a-4d82-ab43-ff5bbcebe5d6,FLIGHTS,FLIGHTS,000030a6d85ad2effe150c581b1735ce243ab4ab,2019-08-05 02:44:42.940000+00:00,LOW_CONVERTING
4965,912,24,1368,d0941e68da8f38151ff86a61fc59f7c5cf9fcaa2,0,flight.list,20397260-1a09-4e54-86a0-e37085858ad1,FLIGHTS,FLIGHTS,000030a6d85ad2effe150c581b1735ce243ab4ab,2019-08-05 02:45:10.157000+00:00,LOW_CONVERTING
14251,912,24,1368,d0941e68da8f38151ff86a61fc59f7c5cf9fcaa2,14,flight.home,b5df6141-a133-49b2-8c55-ff4ed4fe3c58,FLIGHTS,FLIGHTS,000030a6d85ad2effe150c581b1735ce243ab4ab,2019-10-20 17:11:21.537000+00:00,LOW_CONVERTING
14264,912,24,1368,d0941e68da8f38151ff86a61fc59f7c5cf9fcaa2,0,flight.list,f6acbd41-21ad-4eac-b3c0-cba9fadbb87b,FLIGHTS,FLIGHTS,000030a6d85ad2effe150c581b1735ce243ab4ab,2019-10-20 17:12:37.846000+00:00,LOW_CONVERTING


In [ ]:
# convert times to UTC
pi['request.requestedAt'] = pi['request.requestedAt'].dt.tz_convert(None)

# group by publisher user id and get time difference
pi['request.nextRequestedAt'] = pi.groupby('publisherUserId')['request.requestedAt'].shift(-1)
pi['request.lastRequestedAt'] = pi.groupby('publisherUserId')['request.requestedAt'].shift(1)
pi['request.timeDifference'] = (pi['request.requestedAt'] - pi['request.lastRequestedAt']).dt.total_seconds() / 60
pi[['request.requestedAt','request.lastRequestedAt','request.timeDifference']].head()

,request.requestedAt,request.lastRequestedAt,request.timeDifference
8936,2019-10-17 09:54:25.819,NaT,NaN
4946,2019-08-05 02:44:42.940,NaT,NaN
4965,2019-08-05 02:45:10.157,2019-08-05 02:44:42.940,0.453617
14251,2019-10-20 17:11:21.537,2019-08-05 02:45:10.157,110306.189667
14264,2019-10-20 17:12:37.846,2019-10-20 17:11:21.537,1.271817


In [ ]:
# create session ids
pi['sessionId'] = pi.groupby('publisherUserId')['request.timeDifference'].transform(lambda x: x >= 15).cumsum() + 1

In [ ]:
preview_cols = ['publisherUserId','request.requestedAt','request.lastRequestedAt','request.nextRequestedAt','request.timeDifference',
                'sessionId']
pi.loc[pi['publisherUserId'] == '000030a6d85ad2effe150c581b1735ce243ab4ab', preview_cols].reset_index().sort_values('request.requestedAt')

,index,publisherUserId,request.requestedAt,request.lastRequestedAt,request.nextRequestedAt,request.timeDifference,sessionId
0,4946,000030a6d85ad2effe150c581b1735ce243ab4ab,2019-08-05 02:44:42.940,NaT,2019-08-05 02:45:10.157,NaN,1
1,4965,000030a6d85ad2effe150c581b1735ce243ab4ab,2019-08-05 02:45:10.157,2019-08-05 02:44:42.940,2019-10-20 17:11:21.537,0.453617,1
2,14251,000030a6d85ad2effe150c581b1735ce243ab4ab,2019-10-20 17:11:21.537,2019-08-05 02:45:10.157,2019-10-20 17:12:37.846,110306.189667,2
3,14264,000030a6d85ad2effe150c581b1735ce243ab4ab,2019-10-20 17:12:37.846,2019-10-20 17:11:21.537,2019-10-20 17:14:11.924,1.271817,2
4,14315,000030a6d85ad2effe150c581b1735ce243ab4ab,2019-10-20 17:14:11.924,2019-10-20 17:12:37.846,2019-10-20 17:15:04.925,1.567967,2
5,14365,000030a6d85ad2effe150c581b1735ce243ab4ab,2019-10-20 17:15:04.925,2019-10-20 17:14:11.924,2019-10-20 17:15:16.656,0.883350,2
6,14331,000030a6d85ad2effe150c581b1735ce243ab4ab,2019-10-20 17:15:16.656,2019-10-20 17:15:04.925,2019-10-20 17:17:00.160,0.195517,2
7,14403,000030a6d85ad2effe150c581b1735ce243ab4ab,2019-10-20 17:17:00.160,2019-10-20 17:15:16.656,2019-10-20 17:17:49.034,1.725067,2
8,14450,000030a6d85ad2effe150c581b1735ce243ab4ab,2019-10-20 17:17:49.034,2019-10-20 17:17:00.160,2019-10-20 17:24:21.640,0.814567,2
9,14619,000030a6d85ad2effe150c581b1735ce243ab4ab,2019-10-20 17:24:21.640,2019-10-20 17:17:49.034,2019-10-20 17:25:39.265,6.543433,2


In [ ]:
pi.to_hdf(f'{str(DATAFRAMES)}/PageInits.hdf', key='pi', format='table')

## Conversions

In [ ]:
cv = dfs['Conversions'].compute()

In [ ]:
cv = cv.sort_values(['publisherUserId', 'request.requestedAt'])

In [ ]:
cv = cv.drop(columns=['entityId'])

In [ ]:
# fix request date
cv['request.requestedAt'] = pi['request.requestedAt'].dt.tz_convert(None)

In [ ]:
# group by conversions by all columns outside request.requestId / 'request.requestedAt'
cols_to_group = list(set(cv.columns.values) - {'request.requestId', 'request.requestedAt'})
cv = cv.groupby(cols_to_group).agg({'request.requestedAt': 'max', 'request.requestId': 'count'}).reset_index()

In [ ]:
cv.head()

,tripType,browserRequest.screenWidth,browserRequest.screenPixelDepth,travelDateStart,carPickupAirportCode,publisherUserId,pageId,carDropoffAirportCode,destination,travelers,travelDateEnd,origination,browserRequest.screenHeight,cookieSize,orderId,advancePurchaseRangeType,productCategoryType,request.requestedAt,request.requestId
0,ONE_WAY,360,32,2020-01-01 00:00:00+00:00,PER,7d84adfbe42943c4955bb41cbb5eb1f6100b163f,flight.confirmation,SZX,SZX,1,2020-01-01 00:00:00+00:00,PER,720,69,7f1007c8a2ca9067371a2c53b083648c8a68d6de,WEEKDAY_TRAVEL_GREATER_THAN_21_DAYS,FLIGHTS,2019-10-13 05:31:18.203000+00:00,1
1,ONE_WAY,820,24,2019-12-18 00:00:00+00:00,JFK,3523552726bdb5429dd2fec1d63fde64e1330692,flight.confirmation,AMD,AMD,1,2019-12-18 00:00:00+00:00,JFK,615,125,c2298b7e05f288ece87077c28ddfa3cae182f8e8,WEEKDAY_TRAVEL_GREATER_THAN_21_DAYS,FLIGHTS,2019-10-04 12:41:20.292000+00:00,4
2,ONE_WAY,840,24,2019-12-14 00:00:00+00:00,SFO,05210d5ad8aba5cf2474cfc9ae9e149b00fc705a,flight.confirmation,BZE,BZE,1,2019-12-14 00:00:00+00:00,SFO,526,69,0a19d48945becaa184b6d6d2a3e38a7db476fa18,WEEKEND_TRAVEL_GREATER_THAN_21_DAYS,FLIGHTS,2019-08-19 21:46:35.621000+00:00,4
3,ONE_WAY,900,24,2019-09-07 00:00:00+00:00,NYC,fa3cd0ddd38ec5505e4b02b9cbd85cb76df0ccc8,flight.confirmation,EDI,EDI,1,2019-09-07 00:00:00+00:00,NYC,1600,2251,ab8d76d56f53b74e7f92f017c3d02c4b674b40e3,WEEKEND_TRAVEL_LESS_THAN_OR_EQUAL_TO_21_DAYS,FLIGHTS,2019-08-21 18:37:16.623000+00:00,4
4,ONE_WAY,912,24,2019-09-04 00:00:00+00:00,CHI,4297b211f810c00904947b1d5e9a1234cd995936,flight.confirmation,NYC,NYC,1,2019-09-04 00:00:00+00:00,CHI,608,2419,31e82d79133cab94e7b721f764299f06fd15a2af,WEEKDAY_TRAVEL_LESS_THAN_OR_EQUAL_TO_21_DAYS,FLIGHTS,2019-09-01 21:43:39.637000+00:00,4


In [ ]:
# create days from travel columns
cv['daysFromTravel'] = (cv['travelDateStart'] - cv['request.requestedAt']).dt.days

In [ ]:
preview_cols = ['publisherUserId','request.requestedAt','travelDateStart','daysFromTravel']
cv[preview_cols].head()

,publisherUserId,request.requestedAt,travelDateStart,daysFromTravel
0,7d84adfbe42943c4955bb41cbb5eb1f6100b163f,2019-10-13 05:31:18.203000+00:00,2020-01-01 00:00:00+00:00,79
1,3523552726bdb5429dd2fec1d63fde64e1330692,2019-10-04 12:41:20.292000+00:00,2019-12-18 00:00:00+00:00,74
2,05210d5ad8aba5cf2474cfc9ae9e149b00fc705a,2019-08-19 21:46:35.621000+00:00,2019-12-14 00:00:00+00:00,116
3,fa3cd0ddd38ec5505e4b02b9cbd85cb76df0ccc8,2019-08-21 18:37:16.623000+00:00,2019-09-07 00:00:00+00:00,16
4,4297b211f810c00904947b1d5e9a1234cd995936,2019-09-01 21:43:39.637000+00:00,2019-09-04 00:00:00+00:00,2


In [ ]:
cv.loc[cv['publisherUserId'] == '0002eaacc4a7f7c079e90bc88e9d3faea72c9dea', :]

,tripType,browserRequest.screenWidth,browserRequest.screenPixelDepth,travelDateStart,carPickupAirportCode,publisherUserId,pageId,carDropoffAirportCode,destination,travelers,travelDateEnd,origination,browserRequest.screenHeight,cookieSize,orderId,advancePurchaseRangeType,productCategoryType,request.requestedAt,request.requestId,daysFromTravel
12585,ONE_WAY,1680,24,2019-10-19 00:00:00+00:00,FLL,0002eaacc4a7f7c079e90bc88e9d3faea72c9dea,flight.confirmation,SEA,SEA,1,2019-10-19 00:00:00+00:00,FLL,1050,125,29bf57d0e19a38a4bdd12799d2ca5f818d7de878,WEEKEND_TRAVEL_LESS_THAN_OR_EQUAL_TO_21_DAYS,FLIGHTS,2019-10-17 00:42:13.787000+00:00,4,1


In [ ]:
cv = cv.sort_values(['publisherUserId', 'request.requestedAt'])
cv.to_hdf(f'{str(DATAFRAMES)}/Conversions.hdf', key='cv', format='table')

## Client Side Infos

In [ ]:
csi = dd.read_hdf(f'{str(DATAFRAMES)}/ClientSideInfos.hdf', key='df-*')
csi = csi.compute()

In [ ]:
csi.head()

,clientId,clientTime,customDimensions.clickTarget,customDimensions.documentHeight,customDimensions.documentWidth,customDimensions.label,customDimensions.pageX,customDimensions.pageY,customDimensions.selector,customDimensions.viewportHeight,customDimensions.viewportWidth,name,pageId,pageInitializationId,publisherUserId,request.requestedAt
0,3ee55b375ac83d05d90a1516006029101f250cc7,2019-08-20 18:59:56.156000+00:00,div#slider-control.slider-control,4132,1503,FILTERS_DEPARTURE_TIME,304,1530,#filtersSection > section > div.card > article...,655,1520,interaction_monitor_knownClick,flight.list,59bceb8a-5d99-42c3-8ea1-d078142b1f1e,8c40fdf063e02ad3b57a3c99a8348338dd0474da,2019-08-20 19:00:01.254000+00:00
1,3ee55b375ac83d05d90a1516006029101f250cc7,2019-08-20 18:59:59.639000+00:00,div.slider-center,4132,1503,FILTERS_DEPARTURE_TIME,329,1519,#filtersSection > section > div.card > article...,655,1520,interaction_monitor_knownClick,flight.list,59bceb8a-5d99-42c3-8ea1-d078142b1f1e,8c40fdf063e02ad3b57a3c99a8348338dd0474da,2019-08-20 19:00:01.279000+00:00
6,e5fff86f6a9d343557439f4a6af9ea68f84a9e83,2019-08-20 18:59:58.927000+00:00,input#destination1.form-control.lg.ng-pristine...,3180,1831,FLIGHT_DESTINATION,608,277,#flightSearchTemplateScope > div.flight-search...,938,1847,interaction_monitor_knownClick,flight.home,5a08f086-ebbc-4e01-88bd-06caad09b49d,c44e8d70d6c30c671d742ffa9a9e3e9231b5cc60,2019-08-20 19:00:03.815000+00:00
15,e5fff86f6a9d343557439f4a6af9ea68f84a9e83,2019-08-20 19:00:05.270000+00:00,a#logOutBtn,2990,1506,NAV_BAR_LOG_IN,1199,226,#top > div.container > div > div:nth-child(2) ...,738,1522,interaction_monitor_knownClick,flight.list,33dd1c3a-dd32-4d50-9401-5b71f8542b42,636c066e6c78b33d916a71403c6dd2394fb411fa,2019-08-20 19:00:05.583000+00:00
20,e5fff86f6a9d343557439f4a6af9ea68f84a9e83,2019-08-20 19:00:04.455000+00:00,div.ng-binding,3180,1831,FLIGHT_DESTINATION,663,307,#flightSearchTemplateScope > div.flight-search...,938,1847,interaction_monitor_knownClick,flight.home,5a08f086-ebbc-4e01-88bd-06caad09b49d,c44e8d70d6c30c671d742ffa9a9e3e9231b5cc60,2019-08-20 19:00:09.327000+00:00


In [ ]:
csi = csi.sort_values(['publisherUserId', 'request.requestedAt'])

In [ ]:
# drop client time column
csi = csi.drop(columns=['clientId','clientTime','customDimensions.clickTarget','customDimensions.selector','name',
                        'customDimensions.pageX','customDimensions.pageY'])

In [ ]:
# group by everything except request.requestedAt (get max)
cols_to_group = list(set(csi.columns.values) - {'request.requestedAt'})
csi = csi.groupby(cols_to_group).agg({'request.requestedAt': 'max'}).reset_index()

In [ ]:
# create next label
csi['customDimensions.nextLabel'] = csi.groupby('publisherUserId')['customDimensions.label'].shift(-1)

In [ ]:
csi.loc[csi['publisherUserId'] == '87582cf449bcaaef8041a75520c50b3bc4fc5e69', :]

,customDimensions.documentWidth,pageId,customDimensions.viewportHeight,customDimensions.viewportWidth,customDimensions.documentHeight,customDimensions.label,pageInitializationId,publisherUserId,request.requestedAt,customDimensions.nextLabel
1084313,1686,flight.home,781,1707,3269,LOGO,cd0d52c8-60a0-46cb-ac6d-8ba09e56db00,87582cf449bcaaef8041a75520c50b3bc4fc5e69,2019-08-20 01:52:53.977000+00:00,LOGO
1085002,1686,flight.home,831,1707,3068,LOGO,70be7f8b-4dd8-41aa-ba56-2f2f77b869c4,87582cf449bcaaef8041a75520c50b3bc4fc5e69,2019-08-21 03:13:14.273000+00:00,NAV_BAR_LOG_IN
1085004,1686,flight.home,831,1707,3181,NAV_BAR_LOG_IN,70be7f8b-4dd8-41aa-ba56-2f2f77b869c4,87582cf449bcaaef8041a75520c50b3bc4fc5e69,2019-08-20 21:09:42.498000+00:00,NAV_BAR_LOG_IN
1085005,1686,flight.home,831,1707,3181,NAV_BAR_LOG_IN,8226b258-a18a-48dc-9d1e-7e1d6a0ae296,87582cf449bcaaef8041a75520c50b3bc4fc5e69,2019-08-19 22:14:34.586000+00:00,NAV_BAR_LOG_IN
1085008,1686,flight.home,831,1707,3181,NAV_BAR_LOG_IN,fd3871c2-22e9-49c5-bd26-983dc522f39f,87582cf449bcaaef8041a75520c50b3bc4fc5e69,2019-08-21 21:25:49.460000+00:00,FLIGHT_DESTINATION
1085022,1686,flight.home,831,1707,3209,FLIGHT_DESTINATION,ad1ebdcb-8d68-474d-888d-6ab04fd16e5d,87582cf449bcaaef8041a75520c50b3bc4fc5e69,2019-08-21 03:32:43.033000+00:00,FLIGHT_ORIGIN
1085023,1686,flight.home,831,1707,3209,FLIGHT_ORIGIN,ad1ebdcb-8d68-474d-888d-6ab04fd16e5d,87582cf449bcaaef8041a75520c50b3bc4fc5e69,2019-08-21 03:32:36.137000+00:00,NAV_BAR_LOG_IN
1085024,1686,flight.home,831,1707,3209,NAV_BAR_LOG_IN,70be7f8b-4dd8-41aa-ba56-2f2f77b869c4,87582cf449bcaaef8041a75520c50b3bc4fc5e69,2019-08-21 01:15:21.525000+00:00,NAV_BAR_LOG_IN
1085025,1686,flight.home,831,1707,3209,NAV_BAR_LOG_IN,cd0d52c8-60a0-46cb-ac6d-8ba09e56db00,87582cf449bcaaef8041a75520c50b3bc4fc5e69,2019-08-19 22:29:24.833000+00:00,TRIP_TYPE
1085026,1686,flight.home,831,1707,3209,TRIP_TYPE,ad1ebdcb-8d68-474d-888d-6ab04fd16e5d,87582cf449bcaaef8041a75520c50b3bc4fc5e69,2019-08-21 03:32:43.058000+00:00,TRIP_TYPE_ONE_WAY


In [ ]:
csi.to_hdf('../dataframes/ClientSideInfos.hdf', key='csi', format='table')